In [145]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns

In [146]:
telco = pd.read_csv("D:\\MIS 395\\MIS 395_2132300562_Nguyen Thanh Dat\\MIS395_TelcoCustomerChurn.csv")
telco

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,NaN,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [147]:
telco.shape

(7043, 21)

In [148]:
telco.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       5
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       5
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [149]:
telco['MultipleLines'].value_counts()

MultipleLines
No                  3389
Yes                 2968
No phone service     681
Name: count, dtype: int64

In [150]:
telco['PaymentMethod'].value_counts()

PaymentMethod
Electronic check             2363
Mailed check                 1612
Bank transfer (automatic)    1544
Credit card (automatic)      1519
Name: count, dtype: int64

In [151]:
# Kiểm tra số lượng giá trị thiếu (null) trong cột 'TotalCharges'
null_values_totalcharges = telco['TotalCharges'].isnull().sum()

# In kết quả
print(f"Số lượng giá trị null trong cột 'TotalCharges': {null_values_totalcharges}")

Số lượng giá trị null trong cột 'TotalCharges': 0


In [152]:
# Kiểm tra các giá trị không hợp lệ trong 'TotalCharges'
invalid_values = telco[pd.to_numeric(telco['TotalCharges'], errors='coerce').isnull()]
print(invalid_values)

      customerID  gender  SeniorCitizen Partner Dependents  tenure  \
488   4472-LVYGI  Female              0     Yes        Yes       0   
753   3115-CZMZD    Male              0      No        Yes       0   
936   5709-LVOEQ  Female              0     Yes        Yes       0   
1082  4367-NUYAO    Male              0     Yes        Yes       0   
1340  1371-DWPAZ  Female              0     Yes        Yes       0   
3331  7644-OMVMY    Male              0     Yes        Yes       0   
3826  3213-VVOLG    Male              0     Yes        Yes       0   
4380  2520-SGTTA  Female              0     Yes        Yes       0   
5218  2923-ARZLG    Male              0     Yes        Yes       0   
6670  4075-WKNIU  Female              0     Yes        Yes       0   
6754  2775-SEFEE    Male              0      No        Yes       0   

     PhoneService     MultipleLines InternetService       OnlineSecurity  ...  \
488            No  No phone service             DSL                  Yes  ... 

In [153]:
# Chuyển 'TotalCharges' thành số, bỏ qua lỗi (coerce)
telco['TotalCharges'] = pd.to_numeric(telco['TotalCharges'], errors='coerce')

# Điền giá trị thiếu bằng giá trị trung bình
telco['TotalCharges'].fillna(telco['TotalCharges'].mean(), inplace=True)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14732\1581117632.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  telco['TotalCharges'].fillna(telco['TotalCharges'].mean(), inplace=True)


In [154]:
telco['MultipleLines'].fillna(telco['MultipleLines'].mode()[0], inplace=True)
telco['PaymentMethod'].fillna(telco['PaymentMethod'].mode()[0], inplace=True)
missing_values_after = telco.isnull().sum()
print(missing_values_after)

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14732\2292870387.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  telco['MultipleLines'].fillna(telco['MultipleLines'].mode()[0], inplace=True)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14732\2292870387.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

In [ ]:
duplicate_rows = telco[telco.duplicated()]
print(f"Số lượng bản ghi trùng lặp: {duplicate_rows.shape[0]}")

In [ ]:
telco.info()

In [ ]:
telco['SeniorCitizen'] = telco['SeniorCitizen'].map({0: 'No', 1: 'Yes'})

In [ ]:
telco['SeniorCitizen'].info()

In [ ]:
telco['TotalCharges'] = pd.to_numeric(telco['TotalCharges'], errors='coerce')
print(telco['TotalCharges'].dtype)
print(telco['TotalCharges'].head())

In [ ]:
telco.describe()

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(y=telco['tenure'])
plt.title('Boxplot of Tenure')
plt.xlabel('Tenure')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(y=telco['MonthlyCharges'])
plt.title('Boxplot of MonthlyCharges')
plt.xlabel('MonthlyCharges')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(y=telco['TotalCharges'])
plt.title('Boxplot of TotalCharges')
plt.xlabel('TotalCharges')
plt.show()

In [ ]:
Q1 = telco['TotalCharges'].quantile(0.25)
Q3 = telco['TotalCharges'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = telco[(telco['TotalCharges'] < lower_bound) | (telco['TotalCharges'] > upper_bound)]
print(outliers[['customerID', 'TotalCharges']])

In [ ]:
telco[['MonthlyCharges', 'TotalCharges', 'tenure']].hist(bins=20, figsize=(10, 8))
plt.suptitle('Histograms of Numeric Columns')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Pair plot để kiểm tra các mối quan hệ giữa các biến liên quan
sns.pairplot(telco, hue='Churn', diag_kind='kde', height=2.5)
plt.show()

In [ ]:
X = telco.drop(['Churn', 'customerID'], axis=1)  # Loại bỏ cột Churn và customerID
y = telco['Churn']

In [ ]:
X_encoded = pd.get_dummies(X, drop_first=True)  # Drop the first column to avoid multicollinearity

# Kiểm tra lại các cột sau khi mã hóa
print(X_encoded.head())

In [ ]:
X.info()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Tách dữ liệu thành X (đặc trưng) và y (mục tiêu - Churn)
X = X_encoded  # Dữ liệu đã mã hóa
y = telco['Churn']  # Biến mục tiêu

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo mô hình RandomForest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Huấn luyện mô hình
rf_model.fit(X_train, y_train)

# Dự đoán trên dữ liệu kiểm tra
y_pred = rf_model.predict(X_test)

In [ ]:
# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, classification_report

conf_matrix = confusion_matrix(y_test, y_pred)

# Vẽ Confusion Matrix dưới dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# In classification report (Precision, Recall, F1-Score)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

# Dự đoán xác suất của các mẫu (probability) thay vì dự đoán nhãn
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]  # Dự đoán xác suất cho lớp 'Churn' (1)

# Tính ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC: {roc_auc:.2f}")


In [ ]:
pip install scikit-learn

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Khởi tạo mô hình Decision Tree
dt_model = DecisionTreeClassifier(random_state=42)

# Huấn luyện mô hình
dt_model.fit(X_train, y_train)

# Dự đoán
y_pred_dt = dt_model.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred_dt)
print(f"Accuracy of Decision Tree: {accuracy:.2f}")

In [ ]:
# Tính Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_dt)

# Vẽ Confusion Matrix dưới dạng heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Tính Precision, Recall, F1-Score
print("Classification Report:")
print(classification_report(y_test, y_pred_dt))

In [ ]:
# Đường dẫn tới file trên ổ đĩa D
file_path = 'D:/telco_processed.csv'

# Lưu DataFrame 'telco' sau khi xử lý vào file CSV
telco.to_csv(file_path, index=False)

print(f"Dữ liệu đã được xuất ra {file_path}")
